**PROBLEMS**
<br>
Cannot webscrape 
- Niche.com 

Can webscrape: 
- Payscale 

In [2]:
import requests 
import bs4
import pandas as pd

In [10]:
college_rankings = requests.get("https://www.niche.com/colleges/search/best-colleges-for-computer-science/")

In [11]:
soup = bs4.BeautifulSoup(college_rankings.text, 'html.parser')

In [16]:
cards = soup.body.findAll("p")
cards

[<p>
                 Access to this page has been denied because we believe you are using automation tools to browse the
                 website.
             </p>,
 <p>
                 This may happen as a result of the following:
             </p>,
 <p>
                 Please make sure that Javascript and cookies are enabled on your browser and that you are not blocking
                 them from loading.
             </p>,
 <p>
                 Reference ID: #28a127d0-fe8f-11ea-8666-698ea31bcbd7
             </p>,
 <p>
                 Powered by
                 <a href="https://www.perimeterx.com/whywasiblocked">PerimeterX</a>
                 , Inc.
             </p>]

In [ ]:
next_page = 1
while (next_page < 35): 
    # scrape 
    next_page++

In [ ]:
next_page = 2

In [ ]:
url_append = "?page=" + next_page

In [86]:
salary_url = "https://www.payscale.com/college-salary-report/best-schools-by-majors/computer-science"

In [87]:
salaries = requests.get(salary_url)

In [88]:
salaries = requests.get(salary_url)
salary_soup = bs4.BeautifulSoup(salaries.text, 'html.parser')

In [89]:
table = salary_soup.body.findAll("table", class_ = "data-table")
table

[<table class="data-table"><thead><tr><th class="csr-col--rank data-table__header data-table__header--active data-table__header--sortable">Rank<i class="icon-sort-up"></i></th><th class="csr-col--school-name data-table__header data-table__header--sortable">School Name<i class="icon-sort"></i></th><th class="csr-col--right data-table__header data-table__header--sortable"><span class="pxl-tooltip" data-for="earlyCareerMedianPay" data-tip="Median salary for alumni with 0-5 years experience">Early Career Pay<span class="icon-reverse icon-circle"><i class="icon-info"></i></span></span><div class="__react_component_tooltip t070bf58-9169-4ed0-9e9c-14ebab4b516a place-top type-dark" data-id="tooltip" id="earlyCareerMedianPay"><style>
   	.t070bf58-9169-4ed0-9e9c-14ebab4b516a {
 	    color: #fff;
 	    background: #222;
 	    border: 1px solid transparent;
   	}
 
   	.t070bf58-9169-4ed0-9e9c-14ebab4b516a.place-top {
         margin-top: -10px;
     }
     .t070bf58-9169-4ed0-9e9c-14ebab4b516a.p

In [90]:
table = salary_soup.body.findAll("table", class_ = "data-table")
rows = table[0].findAll("tr", class_ = "data-table__row")
rows

[<tr class="data-table__row"><td class="data-table__cell csr-col--rank"><span class="data-table__title">Rank<!-- -->:</span><span class="data-table__value">1</span></td><td class="data-table__cell csr-col--school-name"><span class="data-table__title">School Name<!-- -->:</span><span class="data-table__value"><img src="https://www.payscale.com/content/school_logos/Stanford University_50px.png"/><a href="/research/US/School=Stanford_University/Salary">Stanford University</a></span></td><td class="data-table__cell csr-col--right"><span class="data-table__title">Early Career Pay<!-- -->:</span><span class="data-table__value">$107,400</span></td><td class="data-table__cell csr-col--right"><span class="data-table__title">Mid-Career Pay<!-- -->:</span><span class="data-table__value">$174,800</span></td></tr>,
 <tr class="data-table__row"><td class="data-table__cell csr-col--rank"><span class="data-table__title">Rank<!-- -->:</span><span class="data-table__value">2</span></td><td class="data-t

In [108]:
spans = rows[0].findAll("span")
for sp in spans: 
    print(sp.text)

Rank:
1
School Name:
Stanford University
Early Career Pay:
$107,400
Mid-Career Pay:
$174,800


In [59]:
def parse_school_name(href): 
    school_name = href.split("/research/US/School=")[1]
    school_name = school_name.split("/Salary")[0]
    return school_name 

In [109]:
def parse_career_pay(spans): 
    early_pay = None
    mid_pay = None
    for span in spans: 
        txt = span.text
        if txt.find("$") != -1: 
            if early_pay is None: 
                early_pay = txt.split("$")[1]
            else: 
                mid_pay = txt.split("$")[1]
                break 
    return (early_pay, mid_pay)

In [114]:
def add_rows(salarydf, rows): 
    for row in rows: 
        spans = row.findAll("span")
        title_idx = 0
        for idx, span in enumerate(spans): 
            if span.text == "School Name:": 
                title_idx = idx + 1
                break 
        school_name = spans[title_idx].text     
        #school_name = parse_school_name(link)
        spans = row.findAll("span", class_="data-table__value")
        early_pay, mid_pay = parse_career_pay(spans)
        salarydf = salarydf.append({"school": school_name, "early_pay": early_pay, "mid_pay": mid_pay}, ignore_index = True)
    return salarydf

In [115]:
def get_salaries(page, salarydf, base_url): 
    while page < 26: 
        salary_url = base_url + "page/" + str(page)
        salary_resp = requests.get(salary_url)
        salary_soup = bs4.BeautifulSoup(salary_resp.text, 'html.parser')
        table = salary_soup.body.findAll("table", class_ = "data-table")
        rows = table[0].findAll("tr", class_ = "data-table__row")
        salarydf = add_rows(salarydf, rows)
        page = page + 1 
    return salarydf

In [116]:
base_url = "https://www.payscale.com/college-salary-report/best-schools-by-majors/computer-science/"
salarydf = pd.DataFrame(data = None, columns = ["school", "early_pay", "mid_pay"])
salarydf = get_salaries(1, salarydf, base_url)

In [117]:
salarydf

,school,early_pay,mid_pay
0,Stanford University,"107,400","174,800"
1,Harvey Mudd College,"100,500","172,800"
2,Carnegie Mellon University,"99,000","167,200"
3,University of California-Berkeley,"105,700","166,500"
4,Princeton University,"101,300","162,500"
...,...,...,...
602,Chicago State University,"48,700","86,700"
603,Friends University,"49,300","86,300"
604,Davenport University,"53,000","86,000"
605,Gardner-Webb University,"46,200","80,800"


In [5]:
import bs4
import requests

In [12]:
resp = requests.get("https://oedb.org/rankings/student-faculty-ratio/")

In [13]:
soup = bs4.BeautifulSoup(resp.text, 'html.parser')

In [14]:
p = soup.body.findAll("div")

In [15]:
p

[<div class="wrap">
 <a class="site-logo" href="https://oedb.org" rel="bookmark" title="OEDB.org">OEDB.org</a>
 <nav class="nav header-nav" role="navigation">
 <a class="screen-reader-text skip-link" href="#content">Skip to content</a>
 <ul class="menu main-menu" id="menu-main-nav"><li class="menu-item menu-item-type-post_type menu-item-object-page current-page-ancestor menu-item-has-children parent-item menu-item-17559" id="menu-item-17559"><a href="https://oedb.org/rankings/">Rankings</a>
 <ul class="sub-menu">
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-42820" id="menu-item-42820"><a href="https://oedb.org/rankings/online-accounting-programs/">Accounting</a></li>
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-42821" id="menu-item-42821"><a href="https://oedb.org/rankings/online-business-administration-programs/">Business</a></li>
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-42822" id